In [0]:

import TensorFlow
import Python
import Datasets
import ImageClassificationModels
PythonLibrary.useVersion(3) // Use Python 3.x
// Import some Python libraries
let plt = Python.import("matplotlib.pyplot")
let time = Python.import("time")

func downloadCIFAR10IfNotPresent(to directory: String = ".") {
  let subprocess = Python.import("subprocess")
  let path = Python.import("os.path")
  let filepath = "\(directory)/cifar-10-batches-py"
  let isdir = Bool(path.isdir(filepath))!
  if !isdir {
    print("Downloading CIFAR data...")
    let command = "wget -nv -O- https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz | tar xzf - -C \(directory)"
    subprocess.call(command, shell: true)
  }
}

struct Example: TensorGroup {
  var label: Tensor<Int32>
  var data: Tensor<Float>
}

// Each CIFAR data file is provided as a Python pickle of NumPy arrays
func loadCIFARFile(named name: String, in directory: String = ".") -> Example {
  downloadCIFAR10IfNotPresent(to: directory)
  let np = Python.import("numpy")
  let pickle = Python.import("pickle")
  let path = "\(directory)/cifar-10-batches-py/\(name)"
  let f = Python.open(path, "rb")
  let res = pickle.load(f, encoding: "bytes")

  let bytes = res[Python.bytes("data", encoding: "utf8")]
  let labels = res[Python.bytes("labels", encoding: "utf8")]

  let labelTensor = Tensor<Int64>(numpy: np.array(labels))!
  let images = Tensor<UInt8>(numpy: bytes)!
  let imageCount = images.shape[0]

  // reshape and transpose from the provided N(CHW) to TF default NHWC
  let imageTensor = Tensor<Float>(images
      .reshaped(to: [imageCount, 3, 32, 32])
      .transposed(withPermutations: [0, 2, 3, 1]))

  let mean = Tensor<Float>([0.485, 0.456, 0.406])
  let std  = Tensor<Float>([0.229, 0.224, 0.225])
  let imagesNormalized = ((imageTensor / 255.0) - mean) / std

  return Example(label: Tensor<Int32>(labelTensor), data: imagesNormalized)
}

func loadCIFARTrainingFiles() -> Example {
  let data = (1..<6).map { loadCIFARFile(named: "data_batch_\($0)") }
  return Example(
    label: Raw.concat(concatDim: Tensor<Int32>(0), data.map { $0.label }),
    data: Raw.concat(concatDim: Tensor<Int32>(0), data.map { $0.data })
  )
}

func loadCIFARTestFile() -> Example {
  return loadCIFARFile(named: "test_batch")
}

func loadCIFAR10() -> (
  training: Dataset<Example>, test: Dataset<Example>) {
    let trainingDataset = Dataset<Example>(elements: loadCIFARTrainingFiles())
    let testDataset = Dataset<Example>(elements: loadCIFARTestFile())
    return (training: trainingDataset, test: testDataset)
}



In [0]:

let batchSize = 100
// load the dataset
let dataset = loadCIFAR10()
let testBatches = dataset.test.batched(batchSize)


struct KerasModel: Layer {
    typealias Input = Tensor<Float>
    typealias Output = Tensor<Float>

    var conv1a = Conv2D<Float>(filterShape: (3, 3, 3, 32), padding: .same, activation: relu)
    var conv1b = Conv2D<Float>(filterShape: (3, 3, 32, 32), activation: relu)
    var pool1 = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var dropout1 = Dropout<Float>(probability: 0.25)
    var conv2a = Conv2D<Float>(filterShape: (3, 3, 32, 64), padding: .same, activation: relu)
    var conv2b = Conv2D<Float>(filterShape: (3, 3, 64, 64), activation: relu)
    var pool2 = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var dropout2 = Dropout<Float>(probability: 0.25)
    var flatten = Flatten<Float>()
    var dense1 = Dense<Float>(inputSize: 64 * 6 * 6, outputSize: 512, activation: relu)
    var dropout3 = Dropout<Float>(probability: 0.5)
    var dense2 = Dense<Float>(inputSize: 512, outputSize: 10, activation: identity)

    @differentiable
    func callAsFunction(_ input: Input) -> Output {
        let conv1 = input.sequenced(through: conv1a, conv1b, pool1, dropout1)
        let conv2 = conv1.sequenced(through: conv2a, conv2b, pool2, dropout2)
        return conv2.sequenced(through: flatten, dense1, dropout3, dense2)
    }
}
var model = KerasModel()

// the classic ImageNet optimizer setting diverges on CIFAR-10
// let optimizer = SGD(for: model, learningRate: 0.1, momentum: 0.9)
let optimizer = SGD(for: model, learningRate: 0.001)

print("Starting training...")
print(dataset.training)
for epoch in 1...10 {
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    let trainingShuffled = dataset.training.shuffled(sampleCount:50000,randomSeed: Int64(epoch))
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        let (loss, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
            let logits = model(images)
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }
        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        optimizer.update(&model, along: gradients)
    }

    Context.local.learningPhase = .inference
    var testLossSum: Float = 0
    var testBatchCount = 0
    var correctGuessCount = 0
    var totalGuessCount = 0
    for batch in testBatches {
        let (labels, images) = (batch.label, batch.data)
        let logits = model(images)
        testLossSum += softmaxCrossEntropy(logits: logits, labels: labels).scalarized()
        testBatchCount += 1

        let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels
        correctGuessCount = correctGuessCount
            + Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
        totalGuessCount = totalGuessCount + batchSize
    }

    let accuracy = Float(correctGuessCount) / Float(totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Accuracy: \(correctGuessCount)/\(totalGuessCount) (\(accuracy)) \
        Loss: \(testLossSum / Float(testBatchCount))
        """
    )
}

Starting training...
Dataset<Example>(_handle: TensorFlow.VariantHandle(handle: TensorFlow.TFETensorHandle))
[Epoch 1] Accuracy: 1941/10000 (0.1941) Loss: 2.2519734
[Epoch 2] Accuracy: 2461/10000 (0.2461) Loss: 2.187424
[Epoch 3] Accuracy: 2708/10000 (0.2708) Loss: 2.1010232
[Epoch 4] Accuracy: 2810/10000 (0.281) Loss: 2.042903
[Epoch 5] Accuracy: 2919/10000 (0.2919) Loss: 2.0029104
[Epoch 6] Accuracy: 2999/10000 (0.2999) Loss: 1.9740992
[Epoch 7] Accuracy: 3164/10000 (0.3164) Loss: 1.9436333
[Epoch 8] Accuracy: 3257/10000 (0.3257) Loss: 1.9116424
[Epoch 9] Accuracy: 3377/10000 (0.3377) Loss: 1.8783388
[Epoch 10] Accuracy: 3481/10000 (0.3481) Loss: 1.8418988
